# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



## 말뭉치 다운로드 및 전처리

오픈소스 파이썬 패키지 코포라(Korpora)를 활용해 BPE 수행 대상 말뭉치를 내려받고 전처리합니다. 실습용 말뭉치는 박은정 님이 공개하신 Naver Sentiment Movie Corpus(NSMC)입니다.

다음을 수행해 데이터를 내려받아 `nsmc`라는 변수로 읽어들입니다.

In [2]:
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:57, 254kB/s]                             
[nsmc] download ratings_test.txt: 4.90MB [00:05, 831kB/s]                             


다음을 수행하면 NSMC에 포함된 영화 리뷰(순수 텍스트)들을 지정된 경로에 저장해 둡니다.

In [4]:
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("./2-3_content/train.txt", nsmc.train.get_all_texts())
write_lines("./2-3_content/test.txt", nsmc.test.get_all_texts())

`train.txt`의 앞부분은 다음과 같이 생겼습니다.

In [6]:
!head ./content/train.txt

아 더빙.. 진짜 짜증나네요 목소리
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
너무재밓었다그래서보는것을추천한다
교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
원작의 긴장감을 제대로 살려내지못했다.
별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네
액션이 없는데도 재미 있는 몇안되는 영화
왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?


`test.txt`의 앞부분은 다음과 같이 생겼습니다.

In [7]:
!head ./content/test.txt

굳 ㅋ
GDNTOPCLASSINTHECLUB
뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
지루하지는 않은데 완전 막장임... 돈주고 보기에는....
3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
음악이 주가 된, 최고의 음악영화
진정한 쓰레기
마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다
이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..


## GPT 토크나이저 구축

GPT 계열 모델이 사용하는 토크나이저는 Byte-level Byte Pair Encoding(BBPE)입니다. 우선 어휘집합 구축 결과를 저장해둘 디렉토리를 자신의 구글 드라이브 계정 내 `내 드라이브/nlpbook/bbpe`로 만들어 둡니다.

In [ ]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok=True)

다음을 수행하면 `nsmc` 데이터를 가지고 BBPE 어휘집합을 구축합니다. BBPE 어휘집합 구축에 시간이 걸리니 잠시 기다려주세요. 

In [ ]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("/gdrive/My Drive/nlpbook/bbpe")

위의 코드 수행이 끝나면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/bbpe`)에 `vocab.json`과 `merges.txt`가 생성됩니다. 전자는 바이트 레벨 BPE의 어휘 집합이며 후자는 바이그램 쌍의 병합 우선순위입니다. 

`vocab.json`은 다음과 같이 생겼습니다.

In [ ]:
!cat /gdrive/My\ Drive/nlpbook/bbpe/vocab.json

`merges.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head /gdrive/My\ Drive/nlpbook/bbpe/merges.txt

## BERT 토크나이저 구축

BERT는 워드피스(wordpiece) 토크나이저를 사용합니다. 우선 어휘집합 구축 결과를 저장해둘 디렉토리를 자신의 구글 드라이브 계정 내 `내 드라이브/nlpbook/bbpe`로 만들어 둡니다.


In [ ]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok=True)

다음을 수행하면 BERT 모델이 사용하는 워드피스 어휘집합을 구축할 수 있습니다. 워드피스 어휘집합 구축에 시간이 걸리니 잠시만 기다려주세요.

In [ ]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

위의 코드 수행이 끝나면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 `vocab.txt`가 생성됩니다. `vocab.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head /gdrive/My\ Drive/nlpbook/wordpiece/vocab.txt